In [1]:
import pandas as pd
import numpy as np
import re,os
import datetime

In [63]:
origin= '/pylon5/sy5fp1p/ehanna/logs/hosts/'

In [64]:
pattern = re.compile('(\d+-\d+-\d+)T(\d+:\d+:\d+)-(\d+:\d+)\s(.*?)\.pvt')

In [65]:
nfsTimeouts = []
brNodePath = origin + 'br005.pvt.bridges.psc.edu/'
date = '2017-12-15'
startTime = datetime.datetime.now()
for direc in os.listdir(origin):
    if (direc.startswith('br') or direc.startswith('r')) and direc.endswith('.pvt.bridges.psc.edu'):
        fileName = date+'-'+direc+'.log'
        if fileName in os.listdir(origin+direc+'/'):
            with open(origin+direc+'/'+fileName) as file:
                for line in file:
                    if 'not responding' in line and 'nfs: server ' in line:
                        match = re.search(pattern,line)
                        if match is not None:
                            date = match.group(1)
                            time = match.group(2)
                            Zone = match.group(3)
                            node = match.group(4)
                            nfsTimeouts.append(date+'|'+time+'|'+node)
endTime = datetime.datetime.now()
print(endTime-startTime)

0:00:08.636514


In [67]:
nfsTimeouts

['2017-12-15|05:01:02|r108',
 '2017-12-15|16:33:46|r108',
 '2017-12-15|16:33:46|r108',
 '2017-12-15|03:49:18|r515',
 '2017-12-15|08:03:30|r515',
 '2017-12-15|12:00:54|r515',
 '2017-12-15|12:00:54|r515',
 '2017-12-15|12:24:02|r515',
 '2017-12-15|12:32:49|r515',
 '2017-12-15|03:24:00|r695',
 '2017-12-15|03:24:00|r695',
 '2017-12-15|04:01:59|r695',
 '2017-12-15|01:37:03|r561',
 '2017-12-15|01:37:08|r561',
 '2017-12-15|01:39:02|r561',
 '2017-12-15|01:41:23|r561',
 '2017-12-15|01:41:24|r561',
 '2017-12-15|01:46:51|r561',
 '2017-12-15|01:46:53|r561',
 '2017-12-15|01:57:11|r561',
 '2017-12-15|01:58:01|r561',
 '2017-12-15|02:07:04|r561',
 '2017-12-15|02:08:05|r561',
 '2017-12-15|02:10:11|r561',
 '2017-12-15|02:10:11|r561',
 '2017-12-15|02:10:11|r561',
 '2017-12-15|02:17:38|r561',
 '2017-12-15|02:32:37|r561',
 '2017-12-15|02:33:02|r561',
 '2017-12-15|02:42:51|r561',
 '2017-12-15|02:42:55|r561',
 '2017-12-15|02:43:01|r561',
 '2017-12-15|02:57:36|r561',
 '2017-12-15|02:57:37|r561',
 '2017-12-15|0

In [6]:
sortednfsTimeouts = sorted(nfsTimeouts)

In [7]:
with open(origin+'/br006.pvt.bridges.psc.edu'+'/2018-01-22-br006.pvt.bridges.psc.edu.log') as file:
    count_NR = 0
    count_OK = 0
    count_TO = 0
    count_nfs = 0
    for line in file:
        if 'nfs' in line and 'not responding' in line:
            count_NR+=1
        if 'nfs' in line and 'timed out' in line:
            count_TO+=1
        if 'nfs' in line and 'OK' in line:
            count_OK+=1            
        if 'nfs' in line:
            count_nfs+=1  
    print(count_NR)
    print(count_TO)
    print(count_OK)
    print(count_nfs)

102
31
70
172


In [8]:
uniqueTimestamps = []
for x in sortednfsTimeouts:
    t = x.split('|')[1]
    t = t.split(':')[0]+':'+t.split(':')[1] + ':'+'00'
    if t not in uniqueTimestamps:
        uniqueTimestamps.append(t)

In [9]:
allNodes = []
for x in sortednfsTimeouts:
    t = x.split('|')[2]
    if t not in allNodes:
        allNodes.append(t)

In [10]:
nfsMatrix = pd.DataFrame(0,index=uniqueTimestamps, columns=allNodes)

In [11]:
currentRow = 0
i=0
startTime = datetime.datetime.now()
while i <len(nfsMatrix):
    if currentRow <len(sortednfsTimeouts):
        checkTimestamp = nfsMatrix.index[i]
        checkTimestampDT = datetime.datetime.strptime(checkTimestamp, '%H:%M:%S')
        currentTimeStamp = sortednfsTimeouts[currentRow].split('|')[1]
        currentTimeStampDT = datetime.datetime.strptime(currentTimeStamp, '%H:%M:%S')
        nextTimeStampDT = checkTimestampDT + datetime.timedelta(minutes = 1)
        currentNode = sortednfsTimeouts[currentRow].split('|')[2]
        while currentTimeStampDT>=checkTimestampDT and currentTimeStampDT<nextTimeStampDT:
            nfsMatrix.loc[checkTimestamp,currentNode] +=1
            currentRow +=1
            if currentRow<len(sortednfsTimeouts):
                currentTimeStamp = sortednfsTimeouts[currentRow].split('|')[1]
                currentTimeStampDT = datetime.datetime.strptime(currentTimeStamp, '%H:%M:%S')
                currentNode = sortednfsTimeouts[currentRow].split('|')[2]
            else:
                break
        i+=1
    else:
        break
endTime = datetime.datetime.now()
print(endTime-startTime)

0:00:01.900591


In [12]:
nfsMatrix

,br006,r447,r007,r177,br012,r391,r261,r496,r204,br018,...,r092,r093,r097,r094,r461,r346,r639,r019,r023,r459
03:00:00,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03:01:00,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03:02:00,36,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03:03:00,7,90,40,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03:04:00,2,2,18,75,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03:05:00,5,0,0,60,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03:06:00,3,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03:07:00,7,1,3,2,24,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03:08:00,2,0,1,1,43,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03:09:00,0,1,0,0,41,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
nfsMatrix.to_csv("NFSErrorCount-2018-01-22.csv")

In [14]:
brMatrix = nfsMatrix.loc[:,['br005','br006','br018']]
rMatrix = nfsMatrix.loc[:,[x for x in allNodes if x.startswith('r')]]
brMatrix['brSum'] = brMatrix.sum(axis=1,numeric_only=True)
brMatrix['rSum'] = rMatrix.sum(axis=1,numeric_only=True)
print(len(brMatrix['rSum'][brMatrix['rSum']>0]))
print(len(brMatrix[(brMatrix['rSum']>0) & (brMatrix['brSum']>0)]))
print()

119
81



In [15]:
def makenfsMatrix(dateRange):
    percentComposition = {}
    startDate = datetime.datetime.strptime(dateRange.split(':')[0],"%Y-%m-%d")
    endDate = datetime.datetime.strptime(dateRange.split(':')[1],"%Y-%m-%d")
    currentDate = startDate
    while (currentDate<endDate):
        nfsTimeouts=[]
        startTime = datetime.datetime.now()
        uniqueTimestamps = []
        allNodes = []
        for direc in os.listdir(origin):
            if (direc.startswith('br') or direc.startswith('r')) and direc.endswith('.pvt.bridges.psc.edu'):
                fileName = currentDate.strftime("%Y-%m-%d")+'-'+direc+'.log'
                if fileName in os.listdir(origin+direc+'/'):
                    with open(origin+direc+'/'+fileName) as file:
                        try:
                            for line in file:
                                if 'not responding' in line and 'nfs: server ' in line:
                                    match = re.search(pattern,line)
                                    if match is not None:
                                        date = match.group(1)
                                        time = match.group(2)
                                        Zone = match.group(3)
                                        node = match.group(4)
                                        nfsTimeouts.append(date+'|'+time+'|'+node)
                                        modTime = time[:2]+":"+time[3:5]+":00"
                                        if modTime not in uniqueTimestamps:
                                            uniqueTimestamps.append(modTime)
                                        if node not in allNodes:
                                            allNodes.append(node)
                        except Exception as e:
                            continue
        sortednfsTimeouts = sorted(nfsTimeouts)
        uniqueTimestamps = sorted(uniqueTimestamps)
        nfsMatrix = pd.DataFrame(0,index=uniqueTimestamps, columns=allNodes)
        currentRow = 0
        i=0
        while i <len(nfsMatrix):
            if currentRow <len(sortednfsTimeouts):
                checkTimestamp = nfsMatrix.index[i]
                checkTimestampDT = datetime.datetime.strptime(checkTimestamp, '%H:%M:%S')
                currentTimeStamp = sortednfsTimeouts[currentRow].split('|')[1]
                currentTimeStampDT = datetime.datetime.strptime(currentTimeStamp, '%H:%M:%S')
                nextTimeStampDT = checkTimestampDT + datetime.timedelta(minutes = 1)
                currentNode = sortednfsTimeouts[currentRow].split('|')[2]
                while currentTimeStampDT>=checkTimestampDT and currentTimeStampDT<nextTimeStampDT:
                    nfsMatrix.loc[checkTimestamp,currentNode] +=1
                    currentRow +=1
                    if currentRow<len(sortednfsTimeouts):
                        currentTimeStamp = sortednfsTimeouts[currentRow].split('|')[1]
                        currentTimeStampDT = datetime.datetime.strptime(currentTimeStamp, '%H:%M:%S')
                        currentNode = sortednfsTimeouts[currentRow].split('|')[2]
                    else:
                        break
                i+=1
            else:
                break
        nfsMatrix = nfsMatrix.loc[:,sorted(nfsMatrix.columns)]
        nfsMatrix.to_csv("NFSErrorCount-"+currentDate.strftime("%Y-%m-%d")+".csv")
        rMatrix = nfsMatrix.loc[:,[x for x in allNodes if x.startswith('r')]]
        if 'br005' in allNodes and 'br006' in allNodes and 'br018' in allNodes:
            brMatrix = nfsMatrix.loc[:,['br005','br006','br018']]
            brMatrix['brSum'] = brMatrix.sum(axis=1,numeric_only=True)
            brMatrix['rSum'] = rMatrix.sum(axis=1,numeric_only=True)
            try:
                percentComposition[currentDate.strftime("%Y-%m-%d")] = float(len(brMatrix[(brMatrix['rSum']>0) & (brMatrix['brSum']>0)])/len(brMatrix['rSum'][brMatrix['rSum']>0]))
            except Exception as e:
                percentComposition[currentDate.strftime("%Y-%m-%d")] = 100.00
        endTime = datetime.datetime.now()
        print(str(currentDate)+" " + str(endTime-startTime))
        currentDate = currentDate+datetime.timedelta(days=1)
    return percentComposition

In [16]:
percentComposition = makenfsMatrix('2017-12-15:2018-01-15')

2017-12-15 00:00:00 0:01:08.222730
2017-12-16 00:00:00 0:01:02.744803
2017-12-17 00:00:00 0:01:03.694720
2017-12-18 00:00:00 0:01:07.872213
2017-12-19 00:00:00 0:01:13.102096
2017-12-20 00:00:00 0:01:00.979440
2017-12-21 00:00:00 0:00:58.593935
2017-12-22 00:00:00 0:00:58.453745
2017-12-23 00:00:00 0:00:58.564858
2017-12-24 00:00:00 0:01:05.425879
2017-12-25 00:00:00 0:01:07.772462
2017-12-26 00:00:00 0:00:59.650207
2017-12-27 00:00:00 0:00:58.279946
2017-12-28 00:00:00 0:00:53.760607
2017-12-29 00:00:00 0:00:58.080946
2017-12-30 00:00:00 0:01:04.733324
2017-12-31 00:00:00 0:01:07.982202
2018-01-01 00:00:00 0:00:59.746736
2018-01-02 00:00:00 0:00:57.970168
2018-01-03 00:00:00 0:00:52.471831
2018-01-04 00:00:00 0:00:57.250675
2018-01-05 00:00:00 0:01:00.550971
2018-01-06 00:00:00 0:01:05.541112
2018-01-07 00:00:00 0:00:58.215280
2018-01-08 00:00:00 0:01:10.718872
2018-01-09 00:00:00 0:00:53.210864
2018-01-10 00:00:00 0:00:57.909892
2018-01-11 00:00:00 0:00:59.896843
2018-01-12 00:00:00 

In [59]:
percentComposition = {'2018-01-18': 0.2545454545454545,'2018-01-19': 0.7883435582822086,'2018-01-20': 0.6089385474860335,'2018-01-21': 0.5,'2018-01-22': 0.680672268907563,
'2018-01-23': 0.8676470588235294,
'2018-01-24': 0.42105263157894735,
'2018-01-25': 0.30303030303030304,
'2018-02-04': 0.4528301886792453,
'2018-02-05': 0.0,
'2018-02-06': 0.696969696969697,
'2018-02-16': 100.0,
'2018-02-20': 100.0,
'2018-02-22': 0.0,
'2018-02-26': 100.0,
'2017-12-19': 0.22580645161290322,
'2017-12-29': 0.6666666666666666,
'2018-01-11': 0.8,
'2018-01-14': 0.0
}

In [60]:
nfsPattern = re.compile('NFSErrorCount-(.*?)\.csv')

In [61]:
for direc in os.listdir('./'):
    if direc.startswith('NFSErrorCount-'):
        match = re.search(nfsPattern,direc)
        if match is not None:
            direcDate = match.group(1)
            if direcDate not in percentComposition.keys():
                nfsMatrix = pd.read_csv('./'+direc)
#                 print(nfsMatrix)
                allNodes = nfsMatrix.columns
                rNodes = [x for x in allNodes if x.startswith('r')]
                if ('br005' in allNodes or 'br006' in allNodes or 'br018' in allNodes) and len(rNodes)>0:
                    brNodes = []
                    if 'br005' in allNodes:
                        brNodes.append('br005')
                    if 'br006' in allNodes:
                        brNodes.append('br006')
                    if 'br018' in allNodes:
                        brNodes.append('br018')
                    brMatrix = nfsMatrix.loc[:,brNodes]
                    rMatrix = nfsMatrix.loc[:,rNodes]
                    brMatrix['brSum'] = brMatrix.sum(axis=1,numeric_only=True)
                    brMatrix['rSum'] = rMatrix.sum(axis=1,numeric_only=True)
                    try:
                        percentComposition[direcDate] = float(len(brMatrix[(brMatrix['rSum']>0) & (brMatrix['brSum']>0)])/len(brMatrix['rSum'][brMatrix['rSum']>0]))
                    except Exception as e:
                        percentComposition[direcDate] = 'No overlap between R and Br Nodes'
                else:
                    if 'br005' not in allNodes and 'br006' not in allNodes and 'br018' not in allNodes and len(rNodes)==0:
                        percentComposition[direcDate] = 'Neither R nodes nor Br Nodes'
                    elif 'br005' in allNodes or 'br006' in allNodes or 'br018' in allNodes:
                        percentComposition[direcDate] = 'No R nodes'
                    elif len(rNodes)>0:
                        percentComposition[direcDate] = 'No Br nodes'


In [62]:
percentComposition

{'2017-12-15': 'No Br nodes',
 '2017-12-16': 'No Br nodes',
 '2017-12-17': 0.0,
 '2017-12-18': 0.0,
 '2017-12-19': 0.22580645161290322,
 '2017-12-20': 'No Br nodes',
 '2017-12-21': 'No Br nodes',
 '2017-12-22': 'No Br nodes',
 '2017-12-23': 'No Br nodes',
 '2017-12-24': 'Neither R nodes nor Br Nodes',
 '2017-12-25': 'No Br nodes',
 '2017-12-26': 'No Br nodes',
 '2017-12-27': 'No Br nodes',
 '2017-12-28': 'No Br nodes',
 '2017-12-29': 0.6666666666666666,
 '2017-12-30': 'Neither R nodes nor Br Nodes',
 '2017-12-31': 'No Br nodes',
 '2018-01-01': 'No Br nodes',
 '2018-01-02': 'No Br nodes',
 '2018-01-03': 'No Br nodes',
 '2018-01-04': 'No Br nodes',
 '2018-01-05': 'No Br nodes',
 '2018-01-06': 'No Br nodes',
 '2018-01-07': 'No Br nodes',
 '2018-01-08': 0.0,
 '2018-01-09': 0.0,
 '2018-01-10': 'No R nodes',
 '2018-01-11': 0.8,
 '2018-01-12': 0.015625,
 '2018-01-13': 'No R nodes',
 '2018-01-14': 0.0,
 '2018-01-18': 0.2545454545454545,
 '2018-01-19': 0.7883435582822086,
 '2018-01-20': 0.60893